# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [32]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import pandas as pd

# Work

In [33]:
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap)
driver.get("http://diariooficial.rn.gov.br/dei/dorn3/")

In [38]:
materias = []

def navegar(link, blank=4):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")[:-1]
            print(lista_caminho)
            if blank != 0:
                [lista_caminho.append('') for b in range(blank)]
            lista_caminho.append(subsessoes.text)
            materias.append(lista_caminho)
            print(materias)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")[:-1]
            if blank != 0:
                [lista_caminho.append('') for b in range(blank)]
            lista_caminho.append(link)
            materias.append(lista_caminho)
        return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [35]:
def crawl_diario(link_sessao):
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        #print(sub1)
        for s1 in sub1:
            sub2 = navegar(s1, blank=3)
            #print(sub2)
            if len(sub2) != 0:
                for s2 in sub2:
                    print(s2)
                    sub3 = navegar(s2, blank=2)
                    print(sub3)
                    if len(sub3) != 0:
                        for s3 in sub3:
                            sub4 = navegar(s3, blank=1)
                            print(sub4)
                            if len(sub4) != 0:
                                for s4 in sub4:
                                    sub5 = navegar(s4, blank=0)
                                    print(sub5)
                                    print(materias)
                                    #if len(sub5) == 0:
                                    #    continue

In [39]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')
#lista = navegar(sessoes[7])
#lista = []
for s in sessoes:
    crawl_diario(s)
#print(materias)
#lista

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="666cab94-9abc-4042-bb02-13bb8cda4475", element="f91cf83d-f73b-47be-a9a0-7891cabc1210")>
[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="666cab94-9abc-4042-bb02-13bb8cda4475", element="5d9cd6a3-fd0b-4d8a-bf62-3a76070f8ffb")>]
[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="666cab94-9abc-4042-bb02-13bb8cda4475", element="19825e9d-d0d4-479f-ae2d-41bda9ac3dba")>]
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR']
[['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', 'Nenhuma matéria encontrada nessa data para essa seção.']]
[]
[['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIA

StaleElementReferenceException: Message: The element reference of <a id="dgSubSessoes_ctl03_Linkbutton1" href="javascript:__doPostBack('dgSubSessoes$ctl03$Linkbutton1','')"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


# Teste

Pegar subsessões

In [103]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl02_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
subsessoes = tabela.find_elements_by_tag_name("a")
for s in subsessoes:
    print(s.text)

CONTRATOS, EDITAIS E AVISOS
GOVERNADORIA
SECRETARIAS DE ESTADO


Pegar links html

In [106]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
files = tabela.find_elements_by_tag_name("a")
links = [f.get_attribute("href") for f in files]
links

['http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190921&id_doc=659855',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190921&id_doc=659856']

Caminhar até o fim

In [114]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')

subsessoes_1 = navegar(sessoes[0])
subsessoes_2 = navegar(subsessoes_1[0])
subsessoes_3 = navegar(subsessoes_2[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
subsessoes_5

[]